In [1]:
import sys
from sklearn.metrics import mean_absolute_error
from dataloader import APPLIANCE_ORDER, get_train_test
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable

cuda_av = False
if torch.cuda.is_available():
    cuda_av = True

torch.manual_seed(0)
np.random.seed(0)


# num_hidden, num_iterations, num_layers, p, num_directions = sys.argv[1:6]


class CustomRNN(nn.Module):
    def __init__(self, cell_type, hidden_size, num_layers, bidirectional):
        super(CustomRNN, self).__init__()
        torch.manual_seed(0)

        if bidirectional:
            self.num_directions = 2
        else:
            self.num_directions = 1
        if cell_type == "RNN":
            self.rnn = nn.RNN(input_size=1, hidden_size=hidden_size,
                              num_layers=num_layers, batch_first=True,
                              bidirectional=bidirectional)
        elif cell_type == "GRU":
            self.rnn = nn.GRU(input_size=1, hidden_size=hidden_size,
                              num_layers=num_layers, batch_first=True,
                              bidirectional=bidirectional)
        else:
            self.rnn = nn.LSTM(input_size=1, hidden_size=hidden_size,
                               num_layers=num_layers, batch_first=True,
                               bidirectional=bidirectional)

        self.linear = nn.Linear(hidden_size * self.num_directions, 1)
        self.act = nn.ReLU()

    def forward(self, x):
        pred, hidden = self.rnn(x, None)
        pred = self.linear(pred).view(pred.data.shape[0], -1, 1)
        # pred = self.act(pred)
        # pred = torch.clamp(pred, min=0.)
        pred = self.act(pred)
        pred = torch.min(pred, x)
        return pred


class AppliancesRNN(nn.Module):
    def __init__(self, cell_type, hidden_size, num_layers, bidirectional, num_appliance):
        super(AppliancesRNN, self).__init__()
        self.num_appliance = num_appliance
        self.preds = {}
        self.order = ORDER
        for appliance in range(self.num_appliance):
            if cuda_av:
                setattr(self, "Appliance_" + str(appliance), CustomRNN(cell_type,
                                                                       hidden_size,
                                                                       num_layers,
                                                                       bidirectional).cuda())
            else:
                setattr(self, "Appliance_" + str(appliance), CustomRNN(cell_type,
                                                                       hidden_size,
                                                                       num_layers,
                                                                       bidirectional))

    def forward(self, *args):
        agg_current = args[0]
        flag = False
        if np.random.random() > args[1]:
            flag = True
        # print("Subtracting prediction")
        else:
            pass
        # print("Subtracting true")
        for appliance in range(self.num_appliance):
            # print(agg_current.mean().data[0])
            # print appliance
            # print self.order[appliance]
            # print args[2+appliance]
            #print(getattr(self, "Appliance_" + str(appliance)))
            self.preds[appliance] = getattr(self, "Appliance_" + str(appliance))(agg_current)
            if flag:
                agg_current = agg_current - self.preds[appliance]
            else:
                agg_current = agg_current - args[2 + appliance]

        return torch.cat([self.preds[a] for a in range(self.num_appliance)])

ImportError: No module named 'torch'

In [2]:
from dataloader import APPLIANCE_ORDER, get_train_test
train, test = get_train_test(num_folds=5, fold_num=1)

In [3]:
train_aggregate = train[:, 0, :, :].reshape(-1, 24, 1)

In [15]:
tensor = np.load('../2015-5appliances.numpy.npy')


In [39]:
k = tensor[:, 1:, :, :].copy()

In [40]:
k.mean(axis=3).mean(axis=2).mean(axis=0)

array([ 870.54835262,   86.79575226,   66.9294258 ,   15.4395102 ,
          9.09707779])

In [41]:
k.shape

(68, 5, 112, 24)

In [42]:
k.sum(axis=3).sum(axis=2).sum(axis=0)/k.sum(axis=3).sum(axis=2).sum(axis=0).sum()

array([ 0.83003428,  0.0827564 ,  0.06381463,  0.01472098,  0.00867371])

In [51]:
result = {}
for cell_type in ['GRU', 'LSTM', 'RNN']:
    result[cell_type] = {}
    for hidden_size in [20, 50, 100]:
        result[cell_type][hidden_size] = {}
        for num_layers in [2,4]:
            result[cell_type][hidden_size][num_layers] = {}
            for bidirectional in [True]:
                result[cell_type][hidden_size][num_layers][bidirectional] = {}
                for lr in [1.0]:
                    result[cell_type][hidden_size][num_layers][bidirectional][lr] = {}
                    for iterations in [200, 400, 600, 800]:
                        try:
                            result[cell_type][hidden_size][num_layers][bidirectional][lr][iterations] = np.load("../../baseline/rnn-tree-greedy/{}-{}-{}-{}-{}-{}-0.0.npy".format(cell_type, hidden_size, num_layers,bidirectional, lr, iterations))
                        except:
                            print (cell_type, hidden_size, num_layers,bidirectional, lr, iterations)

GRU 20 4 True 1.0 200
GRU 20 4 True 1.0 400
GRU 20 4 True 1.0 600
GRU 20 4 True 1.0 800
GRU 50 2 True 1.0 800
GRU 50 4 True 1.0 200
GRU 50 4 True 1.0 400
GRU 50 4 True 1.0 600
GRU 50 4 True 1.0 800
GRU 100 2 True 1.0 200
GRU 100 2 True 1.0 400
GRU 100 2 True 1.0 600
GRU 100 2 True 1.0 800
GRU 100 4 True 1.0 200
GRU 100 4 True 1.0 400
GRU 100 4 True 1.0 600
GRU 100 4 True 1.0 800
LSTM 20 4 True 1.0 600
LSTM 20 4 True 1.0 800
LSTM 50 2 True 1.0 200
LSTM 50 2 True 1.0 400
LSTM 50 2 True 1.0 600
LSTM 50 2 True 1.0 800
LSTM 50 4 True 1.0 200
LSTM 50 4 True 1.0 400
LSTM 50 4 True 1.0 600
LSTM 50 4 True 1.0 800
LSTM 100 2 True 1.0 200
LSTM 100 2 True 1.0 400
LSTM 100 2 True 1.0 600
LSTM 100 2 True 1.0 800
LSTM 100 4 True 1.0 200
LSTM 100 4 True 1.0 400
LSTM 100 4 True 1.0 600
LSTM 100 4 True 1.0 800
RNN 20 4 True 1.0 800
RNN 50 4 True 1.0 400
RNN 50 4 True 1.0 600
RNN 50 4 True 1.0 800
RNN 100 2 True 1.0 800
RNN 100 4 True 1.0 200
RNN 100 4 True 1.0 400
RNN 100 4 True 1.0 600
RNN 100 4 True 1

In [63]:
pd.Series(k.item())

fridge hvac dw dr mw    {'mw': 9.09700017504, 'hvac': 157.325275074, '...
dtype: object

In [84]:
for key, v in k.item().items():
    print (key)
    df = (pd.Series(k.item()[key]))

fridge hvac dw dr mw


In [86]:
df.mean()

65.218277583612178

In [89]:
for cell_type in ['GRU', 'LSTM', 'RNN']:
    for hidden_size in [20, 50, 100]:
        for num_layers in [2,4]:
            for bidirectional in [True]:
                for lr in [1.0]:
                    for iterations in [200, 400, 600, 800]:
                        try:
                            k = result[cell_type][hidden_size][num_layers][bidirectional][lr][iterations]
#                             print (k)
                            for key, v in k.item().items():
                                df = (pd.Series(k.item()[key]))
                            print (cell_type, hidden_size, num_layers,bidirectional, lr, iterations, key, df.mean())
                        except:
                            continue
                        

GRU 20 2 True 1.0 200 fridge hvac dr mw dw 87.8883687332
GRU 20 2 True 1.0 400 fridge hvac dr mw dw 70.7448293607
GRU 20 2 True 1.0 600 fridge hvac dr mw dw 63.8863365113
GRU 20 2 True 1.0 800 fridge hvac dr mw dw 59.5332653231
GRU 50 2 True 1.0 200 fridge hvac dw dr mw 85.5916375693
GRU 50 2 True 1.0 400 fridge hvac dw dr mw 75.9527212211
GRU 50 2 True 1.0 600 fridge hvac dw dr mw 73.8051532727
LSTM 20 2 True 1.0 200 fridge hvac dr dw mw 50.9293867059
LSTM 20 2 True 1.0 400 fridge hvac mw dw dr 45.9201925534
LSTM 20 2 True 1.0 600 fridge hvac mw dw dr 45.2295050944
LSTM 20 2 True 1.0 800 fridge hvac mw dw dr 45.0812698817
LSTM 20 4 True 1.0 200 dr dw hvac fridge mw 209.762023734
LSTM 20 4 True 1.0 400 dr dw hvac fridge mw 209.762023734
RNN 20 2 True 1.0 200 fridge dr hvac dw mw 45.70670457
RNN 20 2 True 1.0 400 fridge dw hvac mw dr 45.1265554654
RNN 20 2 True 1.0 600 fridge hvac dw dr mw 45.2444659843
RNN 20 2 True 1.0 800 fridge hvac dw dr mw 45.0487739834
RNN 20 4 True 1.0 200 fridg

In [91]:
result['RNN'][20][2][True][1][800]

array({'fridge hvac dw dr mw': {'mw': 9.0970777913095713, 'hvac': 105.89336602885898, 'fridge': 28.494837124931699, 'dr': 66.31907802555186, 'dw': 15.439510946516529}}, dtype=object)

In [ ]:
def disagg_fold(fold_num, hidden_size, num_layers, bidirectional, lr, num_iterations, p):
    # print (fold_num, hidden_size, num_layers, bidirectional, lr, num_iterations, p)
    torch.manual_seed(0)

    train, test = get_train_test(num_folds=num_folds, fold_num=fold_num)
    train_aggregate = train[:, 0, :, :].reshape(-1, 24, 1)
    test_aggregate = test[:, 0, :, :].reshape(-1, 24, 1)

    out_train = [None for temp in range(len(ORDER))]
    for a_num, appliance in enumerate(ORDER):
        out_train[a_num] = Variable(
            torch.Tensor(train[:, APPLIANCE_ORDER.index(appliance), :, :].reshape((train_aggregate.shape[0], -1, 1))))
        if cuda_av:
            out_train[a_num] = out_train[a_num].cuda()

    loss_func = nn.L1Loss()
    a = AppliancesRNN(cell_type, hidden_size, num_layers, bidirectional, len(ORDER))
    #print(a)
    if cuda_av:
        a = a.cuda()
        loss_func = loss_func.cuda()
    optimizer = torch.optim.Adam(a.parameters(), lr=lr)

    inp = Variable(torch.Tensor(train_aggregate.reshape((train_aggregate.shape[0], -1, 1))).type(torch.FloatTensor),
                   requires_grad=True)
    for t in range(num_iterations):
        inp = Variable(torch.Tensor(train_aggregate), requires_grad=True)
        out = torch.cat([out_train[appliance_num] for appliance_num, appliance in enumerate(ORDER)])
        if cuda_av:
            inp = inp.cuda()
            out = out.cuda()

        params = [inp, p]
        for a_num, appliance in enumerate(ORDER):
            params.append(out_train[a_num])
        # print(params)
        pred = a(*params)

        optimizer.zero_grad()
        loss = loss_func(pred, out)
        if t % 1 == 0:
            print(t, loss.data[0])

        loss.backward()
        optimizer.step()

    test_inp = Variable(torch.Tensor(test_aggregate), requires_grad=False)
    if cuda_av:
        test_inp = test_inp.cuda()

    params = [test_inp, -2]
    for i in range(len(ORDER)):
        params.append(None)
    pr = a(*params)
    pr = torch.clamp(pr, min=0.)
    test_pred = torch.split(pr, test_aggregate.shape[0])
    prediction_fold = [None for x in range(len(ORDER))]

    if cuda_av:
        for appliance_num, appliance in enumerate(ORDER):
            prediction_fold[appliance_num] = test_pred[appliance_num].cpu().data.numpy().reshape(-1, 24)
    else:
        for appliance_num, appliance in enumerate(ORDER):
            prediction_fold[appliance_num] = test_pred[appliance_num].data.numpy().reshape(-1, 24)
    gt_fold = [None for x in range(len(ORDER))]
    for appliance_num, appliance in enumerate(ORDER):
        gt_fold[appliance_num] = test[:, APPLIANCE_ORDER.index(appliance), :, :].reshape(test_aggregate.shape[0], -1,
                                                                                         1).reshape(-1, 24)

    return prediction_fold, gt_fold

In [ ]:
def disagg(hidden_size, num_layers, bidirectional, lr, num_iterations, p):
    from sklearn.metrics import mean_absolute_error
    preds = {}
    gts = {}

    for appliance_num, appliance in enumerate(ORDER):
        preds[appliance] = []
        gts[appliance] = []

    for cur_fold in range(num_folds):
        print (cur_fold, hidden_size, num_layers, bidirectional, lr, num_iterations, p)
        pred, gt = disagg_fold(cur_fold, hidden_size, num_layers, bidirectional, lr, num_iterations, p)
        # pred[pred<0.] = 0.
        for appliance_num, appliance in enumerate(ORDER):
                preds[appliance].append(pred[appliance_num])
                gts[appliance].append(gt[appliance_num])
    for appliance in ORDER:
        print (appliance, mean_absolute_error(np.concatenate(gts[appliance]).flatten(), np.concatenate(preds[appliance]).flatten()))



